# Compute mAP & rotation error

This notebook is used for evlauting model performance by comparing model prediction results and the ground truth using mAP and mAR metrics.

In [26]:
import os
import json
import numpy as np
import cv2
from scipy.spatial.transform import Rotation

def convert_rgb_to_binary(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    return binary

def intersection_over_union(img1, img2):
    intersection = np.logical_and(img1, img2)
    union = np.logical_or(img1, img2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def pixel_wise_metrics(img1, img2):
    tp = np.sum(np.logical_and(img1, img2))
    fp = np.sum(np.logical_and(np.logical_not(img1), img2))
    fn = np.sum(np.logical_and(img1, np.logical_not(img2)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

def process_images(folder_path1, folder_path2):
    img_list1 = os.listdir(folder_path1)
    img_list2 = os.listdir(folder_path2)

    binary_images1 = [convert_rgb_to_binary(cv2.imread(os.path.join(folder_path1, img))) for img in img_list1]
    binary_images2 = [convert_rgb_to_binary(cv2.imread(os.path.join(folder_path2, img))) for img in img_list2]

    precision_values, recall_values, f1_score_values = [], [], []

    for img1, img2 in zip(binary_images1, binary_images2):
        precision, recall, f1_score = pixel_wise_metrics(img1, img2)
        precision_values.append(precision)
        recall_values.append(recall)
        f1_score_values.append(f1_score)

    average_precision = np.mean(precision_values)
    average_recall = np.mean(recall_values)
    average_f1_score = np.mean(f1_score_values)

    print("Average precision:", average_precision)
    print("Average recall:", average_recall)
    print("Average F1 score:", average_f1_score)


def compute_angle_error(gt_euler, pred_euler):
    error = np.abs(gt_euler - pred_euler)
    return error



Change the `folder` and `obj_id` to evaluate different objects in different scenes.

In [27]:
folder = '000015'
obj_id = '29'
folder_path1 = f"/home/zjh/Desktop/AAE_pro/data/t_less/t-less_v2/test_primesense/{folder}/{obj_id}_gt"
folder_path2 = f"/home/zjh/Desktop/AAE_pro/data/t_less/t-less_v2/test_primesense/{folder}/{obj_id}_pred"
process_images(folder_path1, folder_path2)

with open(f"/home/zjh/Desktop/AAE_pro/data/t_less/t-less_v2/test_primesense/{folder}/{obj_id}_gt.json", 'r') as f:
    gt_matrices = json.load(f)

with open(f"/home/zjh/Desktop/AAE_pro/data/t_less/t-less_v2/test_primesense/{folder}/{obj_id}_pred.json", 'r') as f:
    pred_matrices = json.load(f)

angle_errors = []

for key in gt_matrices:
    gt_rot = Rotation.from_matrix(np.array(gt_matrices[key]).reshape(3, 3))
    gt_euler = gt_rot.as_euler('xyz', degrees=True)

    pred_rot = Rotation.from_matrix(np.array(pred_matrices[key]).reshape(3, 3))
    pred_euler = pred_rot.as_euler('xyz', degrees=True)

    angle_error = compute_angle_error(gt_euler, pred_euler)
    angle_errors.append(angle_error)

angle_errors = np.array(angle_errors)
average_errors = np.mean(angle_errors, axis=0)

print("Average angle errors for each axis [roll, pitch, yaw]:", average_errors)

Average precision: 0.8167858033316696
Average recall: 0.8333744197475025
Average F1 score: 0.8129369521179538
Average angle errors for each axis [roll, pitch, yaw]: [105.12268902  24.29510613  82.66137083]
